In [2]:
date_start=date_end = "16.06.2024"#yesterday()#
print(date_start,date_end)

16.06.2024 16.06.2024


In [2]:
import requests, pickle, json, pandas, smtplib, time
from pathlib import Path
from datetime import date, timedelta
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

baseurl="https://ecp.mis66.ru"
api_person = baseurl+"/api/Person"
api_police = baseurl+"/api/Polis"
ecp_searchData = baseurl+"/?c=Search&m=searchData"
cookie_file=Path("cookies.txt")

def make_range(date_start, date_end):
    return date_start+" - " + date_end

def yesterday():
    yesterday = date.today() - timedelta(days=1)
    return yesterday.strftime("%d.%m.%Y")

date_start = "27.10.2025"#yesterday()#
date_end = "27.10.2025"#yesterday()#
collect_range = make_range(date_start,date_end)
send_copy = True
def login():
    print("Logining")
    headers = {
            "Login": "GIBBot1",
            "Password": "Qwerty1!"
    }
    session.request("GET", f"{baseurl}/api/user/login", headers=headers)

def get_person_id_by_snils(snils):
    response = session.request("GET", api_person, params={"PersonSnils_Snils": snils})
    json_response = response.json()
    return json_response['data'][0]['Person_id']

def get_person_police_by_id(person_id):
    response = session.request("GET", api_police, params={"Person_id": person_id})
    json_response = response.json()
    return json_response['data'][0]['Polis_Num']

def get_person_police_by_id_full_response(person_id):
    response = session.request("GET", api_police, params={"Person_id": person_id})
    return response.json()

def get_person_by_id(person_id):
    response = session.request("GET", api_police, params={"Person_id": person_id})
    json_response = response.json()
    return json_response['data'][0]['Polis_Num']

def get_person_by_id_full_response(person_id):
    response = session.request("GET", api_person, params={"Person_id": person_id})
    return response.json()

def find_polices_by_snils():
    for index, row in table.iterrows():
        person_snils=row['СНИЛС']
        if len(person_snils)>10:
            person_id=get_person_id_by_snils(person_snils)
            row['ID']=[person_id]
            person_police=get_person_police_by_id(person_id)
        else:
            personId="Error"
        print(person_snils+","+person_id+","+person_police)
        
def ecp_find_kss_out_daterange(date_range):
    payload = {'PersonPeriodicType_id': '1', 'SearchFormType': 'EvnPS', 'Date_Type':'1', 'EvnPS_disDate_Range':date_range}
    #payload['LpuUnitType_did']=1 # <-- 1 FOR_KSS_ONLY
    print('hello')
    response = session.request("GET", ecp_searchData, params=payload)
    return response.json()

def ecp_find_kss_in_daterange(date_range):
    payload = {'PersonPeriodicType_id': '1', 'SearchFormType': 'EvnPS', 'Date_Type':'1', 'EvnPS_setDate_Range':date_range}
    payload['LpuUnitType_did']=1 # <-- 1 FOR_KSS_ONLY
    print('hello')
    response = session.request("GET", ecp_searchData, params=payload)
    return response.json()

def spid_report_table_to_excel(spid_report_table_data):
    spid_report_table = pandas.DataFrame(spid_report_table_data)
    filename = date_end+"-660017.xlsx"
    spid_report_table.to_excel(filename,index=False)
    return filename
    
def collect_spid_report_table_data(collect_range):
    kss_in_range=ecp_find_kss_in_daterange(collect_range)
    spid_report_table_data=[]
    for pacient in kss_in_range['data']:
        person_response=get_person_by_id_full_response(pacient['Person_id'])
        spid_report_table_data.append({'НомерКарты':pacient['EvnPS_NumCard'],
                                  'Фамилия':pacient['Person_Surname'],
                                  'Имя':pacient['Person_Firname'],
                                  'Отчество':pacient['Person_Secname'],
                                  'ДатаРождения':pacient['Person_Birthday'],
                                  'ДатаПоступления':pacient['EvnPS_setDate'],
                                  'ОтделениеГоспитализации':pacient['LpuSection_Name'],
                                  'СНИЛС':person_response['data'][0]['PersonSnils_Snils'],
                                  'Паспорт':'',
                                  'ПолисСерия':'',
                                  'ПолисНомер':get_person_police_by_id(pacient['Person_id']),
                                  'Эпидномер':'',
                                  'Рекомендации':''})
    return spid_report_table_data

def send_mail_via_smtp(smtp_address, from_address, password, to_address, subject, text, attachment_filepath):
    print('sending message to '+ to_address + ' with attachment ' + attachment_filepath)
    msg = MIMEMultipart()
    msg['From'] = from_address
    msg['To'] = to_address
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    with open(attachment_filepath, "rb") as f:
        attachment = MIMEApplication(f.read(),Name=basename(attachment_filepath))
    attachment['Content-Disposition'] = 'attachment; filename="%s"' % basename(attachment_filepath)
    msg.attach(attachment)
    smtp = smtplib.SMTP(smtp_address, 587)
    smtp.starttls()
    smtp.login(from_address,password)
    result = smtp.sendmail(from_address,to_address,msg.as_string())
    smtp.quit()
    return result


#------------------initialisation--------------------
session = requests.session()
login()

#------------------here-we-are-ready--------------------
table_path_to_file=spid_report_table_to_excel(collect_spid_report_table_data(collect_range))
mail_subject='Поступившие в ГИБ'
mail_text='''
ГАУЗ СО "Городская инфекционная больница город Нижний Тагил"
тел. 8(3435)363518
'''
mail_status={"status":"to_send"}
#mail_status = send_mail_via_smtp('mail.mis66.ru','ntib-public@mis66.ru','72Kji<fc','spid-hospital@mis66.ru', mail_subject, mail_text, table_path_to_file)
while mail_status:
    mail_status = send_mail_via_smtp('mail.mis66.ru','ntib-public@mis66.ru','72Kji<fc','spid-hospital@mis66.ru', mail_subject, mail_text, table_path_to_file)
    print(mail_status)
if send_copy:
    mail_status={"status":"to_send"}
    while mail_status:
        mail_status = send_mail_via_smtp('mail.mis66.ru','ntib-public@mis66.ru','72Kji<fc','ntib-it@mis66.ru', mail_subject, mail_text, table_path_to_file)
        print(mail_status)
print(session.cookies.get_dict())
print('Завершаю работу...')

Logining
hello
sending message to spid-hospital@mis66.ru with attachment 27.10.2025-660017.xlsx
{}
sending message to ntib-it@mis66.ru with attachment 27.10.2025-660017.xlsx
{}
{'PHPSESSID': 'v55bpj36vphele9d5tr5amj5hj', 'route': '799a7afa775c220f49bfebf04e9018f3'}
Завершаю работу...


In [3]:
date_start = yesterday()#"19.06.2024"#
date_end = yesterday()#"22.06.2024"#
collect_range = make_range(date_start,date_end)
    
def login():
    print("Logining")
    headers = {
            "Login": "GIBBot1",
            "Password": "Qwerty1!"
    }
    response=session.request("GET", f"{baseurl}/api/user/login", headers=headers)
    print(response)

login()

Logining
<Response [403]>
